In [ ]:
## html에서 아래 항목을 추출하여 
# 1) 리뷰별 딕셔너리를 목록으로 만들고 
# 2) 전체 내용을 sqlite에 저장

# 구매자
# 평점
# 최대평점
# 상품평
# 구매일
# 크기
# 색상
# 리뷰 이미지 경로

import requests
from bs4 import BeautifulSoup
url = "http://ssacademy.dothome.co.kr/07.html"

response = requests.get(url)
response.encoding = 'utf-8'
html = response.text
soup = BeautifulSoup(html, 'html.parser')

### [각 반복되는 항목을 구분하는 패턴 찾기]
# 예) <article data-review-id="rev1" class="review-item">

# 1. 가장 상위 태그 식별: 뷰티풀숲 작업에서 반복적인 패턴 중 가장 상위 태그를 찾아냅니다.

# 2. CSS 셀렉터 지정: 식별한 상위 태그를 기준으로 CSS 셀렉터를 설정합니다.

# 3. 반복문 사용: 설정한 CSS 셀렉터를 사용하여 반복문을 통해 내부 정보를 추출합니다.

# 4. select는 하위 요소 중에서 특정 요소(태그)를 찾을때 사용

# 5. 자기 자신의 요소(태그)의 속성을 찾을때는 []를 사용

#reviews = soup.select('article.review-item')
reviews = soup.select('[data-review-id]')

data = []

for review in reviews:
    # print(review)
    # print("-"*70)
    reviewer = review.select_one('#name').text
    rating_star = review.select_one('.stars').text
    review_text = review.select_one('.review-text').text
    p_date = review.select_one('time')['datetime']
    #p_date = review.select_one('time').text.split(':')[-1].strip()
    size = review.select_one('.size').text.split(':')[-1].strip()
    color = review.select_one('.color').text.split(':')[-1].strip()

    imgs = review.select(".review-images img")

    images = [img['src'] for img in imgs]
    # images = []
    # for img in imgs:
    #     images.append(img['src'])
    rating = rating_star.count('★')
    rating_max = len(rating_star.strip())

    # print(reviewer,rating_star,rating, rating_max,review_text,p_date, size, color, images, sep=" | ")    

    review_one = {
        'reviewer': reviewer,
        'rating': rating,
        'rating_max': rating_max,
        'review_text': review_text,
        'p_date': p_date,
        'size': size,
        'color': color,
        'images': images
    }

    data.append(review_one)

    # print(review_one)
    # print("-"*100)

data

[{'reviewer': '구매자1',
  'rating': 4,
  'rating_max': 5,
  'review_text': '상품이 매우 좋습니다.',
  'p_date': '2024-01-15',
  'size': 'L',
  'color': 'Black',
  'images': ['review1.jpg', 'review2.jpg']},
 {'reviewer': '구매자2',
  'rating': 5,
  'rating_max': 5,
  'review_text': '정말 기대 이상이었습니다!',
  'p_date': '2024-02-10',
  'size': 'M',
  'color': 'Red',
  'images': ['review3.jpg', 'review4.jpg']},
 {'reviewer': '구매자3',
  'rating': 3,
  'rating_max': 5,
  'review_text': '가격 대비 괜찮은 제품입니다.',
  'p_date': '2024-03-05',
  'size': 'S',
  'color': 'Blue',
  'images': ['review5.jpg', 'review6.jpg']}]

In [16]:
import sqlite3
conn = sqlite3.connect('product_reviews.db')

### DDL
create_table_sql = '''
CREATE TABLE IF NOT EXISTS product_reviews (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    reviewer TEXT,
    rating INTEGER,
    rating_max INTEGER,
    review_text TEXT,
    purchase_date TEXT,
    size TEXT,
    color TEXT,
    images TEXT
);
'''
cursor = conn.cursor()
cursor.execute(create_table_sql)
conn.commit()

for info in data:
    #print(info)
    insert_query = '''
        insert into product_reviews (reviewer,rating,rating_max,review_text,purchase_date,size,color,images) values (?,?,?,?,?,?,?,?)
    '''

    insert_data = (info['reviewer'],info['rating'],info['rating_max'],info['review_text'],info['p_date'],info['size'],info['color'],','.join(info['images']))

    cursor.execute(insert_query, insert_data)

conn.commit()

cursor.close()
conn.close()


### selenium

In [32]:
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

## 크롬 옵션 설정
chrome_options = Options()
#chrome_options.add_argument('--headless')  # 헤드리스 모드
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

## WebDriver 설정
service = Service('chromedriver.exe') 
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get('https://www.python.org')
print("페이지 제목:", driver.title)

search_box = driver.find_element(By.NAME, 'q')
search_box.send_keys('java')
#search_box.send_keys(Keys.ENTER)
search_box.submit()

driver.quit()

페이지 제목: Welcome to Python.org


### webdriver.Chrome 자동 install sample
- webdriver_manager의 ChromeDriverManager를 사용하면, 크롬 드라이버(예: chromedriver.exe)를 직접 다운로드해서 별도로 관리할 필요가 없습니다.
- 즉, 크롬 브라우저만 설치되어 있으면, 크롬 드라이버는 자동으로 다운로드 및 설치·업데이트되어 경로 지정 없이 바로 사용할 수 있습니다.
- 드라이버는 기본적으로 캐시에 저장되며, 필요시 자동으로 업데이트됩니다

In [ ]:
!pip install selenium
!pip install webdriver-manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# Chrome 옵션 설정
chrome_options = Options()
# chrome_options.add_argument('--headless')  # 헤드리스 모드로 실행하려면 주석 해제
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Chrome 드라이버 설정
#service = Service('chromedriver.exe') # 직접 클롬드라이버 설치 버전
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)


# 테스트
driver.get('https://www.python.org')
print("페이지 제목:", driver.title)

driver.quit()


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

def setup_chrome_driver():
    # Chrome 옵션 설정
    chrome_options = Options()
    # chrome_options.add_argument('--headless')  # 헤드리스 모드로 실행하려면 주석 해제
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    # Chrome 드라이버 설정
    #service = Service('chromedriver.exe') # 직접 크롬드라이버 설치 버전 (옛날 방식)

    # 크롬드라이버 자동 설치 버전 (최근 방식)
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    return driver

def main():
    try:
        driver = setup_chrome_driver()
        driver.get('https://www.python.org')
        print("페이지 제목:", driver.title)
    except Exception as e:
        print(f"에러 발생: {e}")        
    finally:
        # 브라우저 종료
        driver.quit()
        
if __name__ == "__main__":
    main()

페이지 제목: Welcome to Python.org


In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Chrome 드라이버 설정
#service = Service('chromedriver.exe') # 직접 클롬드라이버 설치 버전
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

url = "https://www.tistory.com/?category=living"
driver.get(url)

time.sleep(5)

# find_elements == select
# find_element == select_one

#items = soup.select('')
items = driver.find_elements(By.CSS_SELECTOR,'div.item_group')

posts = []

for item in items:
    #print(item)
    title = item.find_element(By.CSS_SELECTOR, '.tit_cont').text.strip()
    desc = item.find_element(By.CSS_SELECTOR, '.wrap_desc').text.strip()
    author = item.find_element(By.CSS_SELECTOR, '.link_blog .txt_g').text.strip()

    posts.append({
        'title': title,
        'description': desc,
        'author': author
    })
    #print(title)

posts


[{'title': '진서연 야채찜 만들기(연예인 다이어트 식단 관리 따라하기, 건강한 다이어트, 맛있는 다이어트, 배부른 다이어트, 드레싱 레시피, 소스 레시피, 진세연 야채찜, 항산화 항암 성분 가득, 야채 요리 추천)',
  'description': '와우. 시작하기 앞서 급 하소연할 것이 있다.. 글을 다 썼는데 지워져서 다시 처음부터 쓰는 중이다. 열심히 임시저장을 눌렀으나 저장되지 않았다... 헤헤헤헷... 웃으며 열심히 다시 쓰는중(해탈) 어쨌든 오늘의 주제는 건강에도 좋고 포만감도 좋고, 살도 뺄 수 있는 야채찜 만들기이다. 뚱이어머니와 딸도 정말 자주 먹고 있는 음식이고, 특히 얼마 전 TV에 진서연님이 레시피를 공개해서 더욱더 유명해진 일명 진서연 아채찜 만들기 포스팅을 지금부터 시작한다.(야채덕후들 다이어터들 다 모여모여) 짜잔, 새롭게 도입한 목차! 아래 목차를 통해서 원하는 부분으로 바로 쩜프할 수 있으니 많은 애용바람. 목 차 1. 야채찜 재료 2. 드레싱(소스) 재료 3. 야채찜 레시피 4. 드레싱(소스)..',
  'author': '뚱이어머니의 하루'},
 {'title': '한복 갓 노리게 악세사리가 뜨는 이유? 케이팝 데몬 헌터스 열풍!',
  'description': "요즘 SNS를 뜨겁게 달구고 있는 케이팝 데몬 헌터스(K-Pop Demon Hunters)를 아시나요? 이 작품이 단순한 애니메이션을 넘어서 전 세계적으로 한국 문화, 특히 한복에 대한 관심을 폭발적으로 증가시키고 있어요!케이팝 데몬 헌터스란?케이팝 데몬 헌터스는 소니 픽처스 애니메이션에서 제작한 작품으로, K-팝 걸그룹이 악마를 물리치는 스토리를 그린 애니메이션입니다. 하지만 이 작품의 진짜 매력은 한국 전통문화를 매력적으로 녹여낸 부분에 있어요.작품 속에서 등장인물들이 한복을 입고 공연하는 장면들이 나오면서, 전 세계 시청자들이 한복의 아름다움에 매료되고 있답니다. 실제로 한복 체험 예약이 전월 대비 30% 급증할 정도로 한복 열풍이 불고 있습니다.지난